# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [2]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
!pip install folium

# Import API key
from api_keys import geoapify_key

In [3]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,aral,40.5156,81.2636,77.92,41,1,19.42,CN,1720464836
1,1,puerto natales,-51.7236,-72.4875,41.45,75,20,5.75,CL,1720464538
2,2,grytviken,-54.2811,-36.5092,29.12,74,7,10.45,GS,1720464839
3,3,adamstown,-25.0660,-130.1015,67.66,64,100,19.55,PN,1720464675
4,4,la punta,-33.1837,-66.3127,50.50,28,0,10.16,AR,1720464842


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [5]:
%%capture --no-display

# Configure the map plot
# YOUR CODE HERE

# Display the map
# YOUR CODE HERE

In [6]:
import folium

# Create a map centered around the average coordinates
map_center = [city_data_df['Lat'].mean(), city_data_df['Lng'].mean()]
city_map = folium.Map(location=map_center, zoom_start=2)

# Add points to the map
for index, row in city_data_df.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Humidity'] / 10, 
        popup=f"{row['City']}: {row['Humidity']}% Humidity",
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(city_map)

# Save the map to an HTML file
city_map.save('city_humidity_map.html')

city_map


### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [8]:
import pandas as pd
import folium


# Filter the DataFrame for cities with max temp between 70 and 90 degrees and humidity between 50 and 80%
filtered_city_data_df = city_data_df[
    (city_data_df['Max Temp'] >= 70) & 
    (city_data_df['Max Temp'] <= 90) & 
    (city_data_df['Humidity'] >= 50) & 
    (city_data_df['Humidity'] <= 80)
]

# Drop rows with null values
filtered_city_data_df = filtered_city_data_df.dropna()

# Create a map centered around the average coordinates of the filtered data
map_center = [filtered_city_data_df['Lat'].mean(), filtered_city_data_df['Lng'].mean()]
city_map = folium.Map(location=map_center, zoom_start=2)

# Add points to the map for the filtered data
for index, row in filtered_city_data_df.iterrows():
    folium.CircleMarker(
        location=[row['Lat'], row['Lng']],
        radius=row['Humidity'] / 10,  # Adjust the radius for better visibility
        popup=f"{row['City']}: {row['Humidity']}% Humidity",
        color='blue',
        fill=True,
        fill_color='blue'
    ).add_to(city_map)

# Save the map to an HTML file
city_map.save('filtered_city_humidity_map.html')

# Display the map
filtered_city_data_df


,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
8,8,ebon,4.5820,168.6964,83.39,74,41,10.60,MH,1720464848
9,9,val-d'or,48.1002,-77.7828,78.89,53,75,8.05,CA,1720464849
13,13,wailua homesteads,22.0669,-159.3780,76.75,75,100,16.11,US,1720464526
27,27,georgetown,5.4112,100.3354,80.55,78,20,2.30,MY,1720464760
28,28,red wing,44.5625,-92.5338,80.78,58,2,4.00,US,1720464877
...,...,...,...,...,...,...,...,...,...,...
568,568,yinchuan,38.4681,106.2731,72.81,50,97,5.77,CN,1720465581
570,570,praia da vitoria,38.7333,-27.0667,80.85,69,20,15.28,PT,1720465584
574,574,bachi-yurt,43.2200,46.2000,81.93,54,72,5.35,RU,1720465590
580,580,hola,51.5377,23.1902,73.58,77,9,3.44,PL,1720465597


In [9]:
city_map

### Step 3: Create a new DataFrame called `hotel_df`.

In [11]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = filtered_city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
8,ebon,MH,4.5820,168.6964,74,
9,val-d'or,CA,48.1002,-77.7828,53,
13,wailua homesteads,US,22.0669,-159.3780,75,
27,georgetown,MY,5.4112,100.3354,78,
28,red wing,US,44.5625,-92.5338,58,
...,...,...,...,...,...,...
568,yinchuan,CN,38.4681,106.2731,50,
570,praia da vitoria,PT,38.7333,-27.0667,69,
574,bachi-yurt,RU,43.2200,46.2000,54,
580,hola,PL,51.5377,23.1902,77,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [27]:
radius = 10000
params = {
    "apiKey": geoapify_key,
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # Get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make an API request using the params dictionary
    response = requests.get(base_url, params=params)
    
    # Check the response status
    if response.status_code != 200:
        print(f"Error: Received status code {response.status_code} for city {row['City']}")
        continue

    # Convert the API response to JSON format
    name_address = response.json()


    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Save the DataFrame to a CSV file
hotel_df.to_csv('hotel_data.csv', index=False)

# Display the DataFrame with hotel names
hotel_df.head()

Starting hotel search
ebon - nearest hotel: No hotel found
val-d'or - nearest hotel: Quality Inn & Suites
wailua homesteads - nearest hotel: Hilton Garden Inn Kauai Wailua Bay
georgetown - nearest hotel: Page 63 hostel
red wing - nearest hotel: The Saint James Hotel
albany - nearest hotel: No hotel found
ketchikan - nearest hotel: Cape Fox Lodge
yaren - nearest hotel: Aiwo Hotel
la'ie - nearest hotel: No hotel found
puerto lempira - nearest hotel: Hotel Yu Baiwan
yellowknife - nearest hotel: No hotel found
sabang - nearest hotel: Sabang Hill
minna - nearest hotel: Haske Main Hotel
kill devil hills - nearest hotel: Mariner Days Inn & Suites
guelmim - nearest hotel: فندق الملتقى
colonia - nearest hotel: Wasserturm Hotel Cologne
tofol - nearest hotel: Nautilus Resort
callao - nearest hotel: Casa Ronald
bel ombre - nearest hotel: Villa La Cachette
pariaman - nearest hotel: HOTEL NAN TONGGA
sokoni - nearest hotel: No hotel found
san marcos de colon - nearest hotel: Hotel
eyl - nearest hotel

,City,Country,Lat,Lng,Humidity,Hotel Name
8,ebon,MH,4.5820,168.6964,74,No hotel found
9,val-d'or,CA,48.1002,-77.7828,53,Quality Inn & Suites
13,wailua homesteads,US,22.0669,-159.3780,75,Hilton Garden Inn Kauai Wailua Bay
27,georgetown,MY,5.4112,100.3354,78,Page 63 hostel
28,red wing,US,44.5625,-92.5338,58,The Saint James Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [28]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng", 
    "Lat", 
    geo = True, 
    size = "Humidity",
    scale = 1,
    color = "City",
    alpha = 0.5,
    tiles = "OSM",
    frame_width = 650,
    frame_height = 475,
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)